In [7]:
#inspiration https://medium.com/lets-talk-ml/guide-to-your-very-own-a-i-virtual-assistant-in-python-d8120ff228a8

#!pip install pyttsx3
#!pip install speechRecognition
#!pip install pyaudio
#!pip install wikipedia
#pip install googletrans  virkede ikke
#pip install googletrans==4.0.0-rc1  # new update der virker
# #not working !pip install ecapture



In [8]:

import speech_recognition as sr
import pyttsx3
from gtts import gTTS
import playsound
import pydub

import datetime
import wikipedia
import webbrowser
import os
import time
import subprocess
#from ecapture import ecapture as ec
import wolframalpha
import json
import requests
import googletrans

#own modules
import send_sms 
import assistant_utils


In [9]:
#setting up folder structure
path_0 = os.getcwd()
path_archive = os.path.join(path_0,'archive')
try:
    os.mkdir(path_archive)
except:
    pass

In [10]:
#utils

#drop file if exists
def drop_if_exists(file_name):
    try:
        os.remove(file_name)
    except:
        pass

#convert wav to 
# def wav2mp3(wav_file):
#     mp3_file = os.path.splitext(wav_file)[0] + '.mp3'
#     sound = pydub.AudioSegment.from_wav(wav_file)
#     print(mp3_file)
#     sound.export(mp3_file, format="mp3")
#     os.remove(wav_file)


In [11]:
#init webbrowser
webbrowser.register('chrome',
	None,
	webbrowser.BackgroundBrowser("C://Program Files (x86)//Google//Chrome//Application//chrome.exe"))
#webbrowser.get('chrome').open(url)


#init takeCommand
#https://cloud.google.com/speech-to-text/docs/languages
lang_dk = 'da-DK'
lang_no = 'no-NO'
lang_eng_GB = 'en-GB'
lang_eng_US = 'en-US'
lang_viet = 'vn-VN'


In [12]:
#VOICE 
#import pyttsx3

def set_voice(gender=0, rate=125,volume=.7):
    engine = pyttsx3.init('sapi5')

    #voice setttings
    #engine.setProperty('voice', voices[gender].id) #male = 0, female = 1
    engine.setProperty('volume',volume)    # setting up volume level  between 0 and 1
    engine.setProperty('rate', rate)     # setting up new voice rate
    return engine
    #engine = set_voice(gender=0, rate=250, volume=.3)


def get_voice_settings(engine):
    volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
    rate = engine.getProperty('rate')   # getting details of current speaking rate
    voices = engine.getProperty('voice') #getting details of the current voice
    print (volume, rate, voices)  
    #get_voice_settings(engine)
    

def speak(text):
    engine.say(text) 
    engine.runAndWait() #Without this command, speech will not be audible to us.
    #speak("Hello World! Hope you all are doing well.")    


#init voice
engine = set_voice(gender=0, rate=185, volume=.4)


In [19]:
#VOICE GTTS

#saves phrase tp mmp3 and plays mp3
def speak_gtts(text, file_name='phrase.mp3'):
    try:
        os.remove(file_name)
    except:
        pass
    tts = gTTS(text=text, lang='en',slow=False)
    tts.save(file_name)
    playsound.playsound(file_name)
    #speak_gtts("hello ")

#saves text to mp3 and plays mp3 on defined language. Can also play english with french accent
def speak_gtts_language(text="hello", lang="en",file_name='translated_phrase.mp3'):
    try:
        os.remove(file_name)
    except:
        pass
    tts = gTTS(text=text, lang=lang,slow=False)
    
    tts.save(file_name)
    playsound.playsound(file_name)
# speak_gtts_language(text="Good day I am Rocky", lang='fr')

In [20]:
#Speech Recognition
#import speech_recognition as sr

#https://cloud.google.com/speech-to-text/docs/languages
#lang_dk = 'da-DK'
#lang_no = 'no-NO'
#lang_eng_GB = 'en-GB'
#lang_eng_US = 'en-US'
#lang_viet = 'vn-VN'


def takeCommand(lang='en-GB'):
    #It takes microphone input from the user and returns string output
    query=''
    wav_file = 'query_audio.wav'
    drop_if_exists(wav_file)

    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=0.2)
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source) 

        #save audio as wave
        with open(wav_file,'wb') as f:
            f.write(audio.get_wav_data())
        #wav2mp3(wav_file)
    
    try:
        print("Recognizing...")    
        query = r.recognize_google(audio, language=lang)#Using google for voice recognition.
        print(f"User said: {query}\n")  #User query will be printed.
    
    except Exception as e:
        print(e) # use only if you want to print the error!
        print("Say that again please...")   #Say that again will be printed in case of improper voice 
        return "None" #None string will be returned
        
    return query.lower()
    #query=takeCommand()
    #query=takeCommand(lang=lang_dk)
  

In [22]:
# functions related to assistant tasks
def show_commands():
    print('"Hello Rocky" - wakes up assistant')
    print('"note to self"')
    print('"send email"')
    print('"send text"')
    print('"open youtube"')
    print('"search google"')
    print('"ask wikipedia"')
    print('"ask wolfram"')
    print('"translate from"')
    print('"record audio"')
    print('"record video"')
    print('"take picture"')
    print("take screenshot")
    print('"describe picture"')
    print('"What is the time?"')

#show_commands()

In [23]:
#text translation

from googletrans import Translator 
import pandas as pd

translator = Translator()
languages = googletrans.LANGUAGES
df_lang = pd.DataFrame(list(languages.items()),columns = ['lang','language']) #dict to df

def lang_abb(lang):
    lang_abb=df_lang['lang'].loc[df_lang.language==lang].values
    #print(lang_abb)
    try:
        src=lang_abb[0]
    except:
        src='' 
    return src


def translate_text(phrase="Hello", src="", dest="en"): 
    if len(src)<2:
        detected = translator.detect(phrase)
        src = detected.lang
    translated = translator.translate(phrase,src=src,dest=dest)     
    print(translated.text)
    print(src, dest)
    trans_text = translated.text
    return trans_text


def translate_spoken():
    speak("what do you want me to translate?")
    phrase=takeCommand()
    
    speak("To what language?")
    q_dest=takeCommand()
    dest=lang_abb(q_dest)
    
    try:
        trans_text=translate_text(phrase, dest=dest)
        speak_gtts_language(text=trans_text, lang=dest)
    except:
        speak("I did not get that....here it is in korean")
        trans_text=translate_text(phrase, dest='ko')
        speak_gtts_language(text=trans_text, lang='ko')


def translate_from():
    speak("From what language do you want me to translate?")
    q_src=takeCommand()
    src=lang_abb(q_src)
    print("from " + src)

    speak("..and to what language?")
    q_dest=takeCommand()
    dest=lang_abb(q_dest)
    print('to ' + dest)
    
    speak("Let me hear it?")
    phrase=takeCommand(lang=src[0])
    
    try:
        trans_text=translate_text(phrase, src=src[0], dest=dest[0])
        speak_gtts_language(text=trans_text, lang=dest[0])
    except:
        speak("I did not get that....here it is in korean")
        trans_text=translate_text(phrase, dest='ko')
        speak_gtts_language(text=trans_text, lang='ko')


In [24]:
 
def take_dictate():
    speak("What language")
    q_src=takeCommand()
    #q_src='french'
    src=lang_abb(q_src)
    if src!='':
            speak(q_src + "..it is" )
    if src=='':
        speak("Can you repeat that")
        q_src=takeCommand()
        src=lang_abb(q_src)
        if src!='':
            speak(q_src + "..it is" )
        elif src=='':
            speak("I am confused...Lets take it in english")
            src='en'
        else:
            speak(q_src + "..it is" )

    phrase=takeCommand(lang=src)
    return(phrase)

'''
def take_dictate_2():
    #try:
     #   trans_text=translate_text(phrase, dest=dest[0])
      #  speak_gtts_language(text=trans_text, lang=dest[0])

    if "danish language" in query:
        speak("Danish it is. You can dictate in danish now.")
        query_lang=takeCommand(lang=lang_dk)
    elif "norwegian language" in query:
        query_lang=takeCommand(lang=lang_no)
        speak("Norwegian it is. You can dictate in norwegian now.")
    elif "vietnamese language" in query:
        query_lang=takeCommand(lang=lang_viet)
        speak("Vietnamese it is. You can dictate in vietnamese now.")
    else:
        query_lang=query
    return query_lang
'''    #query = queryLanguage(query)


'\ndef take_dictate_2():\n    #try:\n     #   trans_text=translate_text(phrase, dest=dest[0])\n      #  speak_gtts_language(text=trans_text, lang=dest[0])\n\n    if "danish language" in query:\n        speak("Danish it is. You can dictate in danish now.")\n        query_lang=takeCommand(lang=lang_dk)\n    elif "norwegian language" in query:\n        query_lang=takeCommand(lang=lang_no)\n        speak("Norwegian it is. You can dictate in norwegian now.")\n    elif "vietnamese language" in query:\n        query_lang=takeCommand(lang=lang_viet)\n        speak("Vietnamese it is. You can dictate in vietnamese now.")\n    else:\n        query_lang=query\n    return query_lang\n'

In [25]:
#TEST FFEATURES

#import send_sms 

print("")
def test_features():
    send_sms.send_sms(4527285100, 'ROCKY','skru ned')

#translate_to('"Hej jeg hedder Rocky")
def test_translate():
    while True:
        print()
        translate_spoken()
        t=input("press return to go again")

def test_translate_from():
    while True:
        print()
        translate_from()
        t=input("press return to go again")

#test_translate()
#test_translate_from()

In [26]:

#receiver of message
def get_receiver(query):
    if "celina" in query or "selina" in query:
        name = 'Celina'
        email = "ccjeremiassen@gmail.com"
        mobil_no= 4571706803
    elif "magnus" in query:
        name = 'Magnus'
        email = 'mcjeremiassen@gmail.com'
        mobil_no= 4531751171
    elif "to self" in query:
        name = 'Lasse'
        email = 'essalj@gmail.com'
        mobil_no= 4527285100
    else:
         name = ''
         email = ''
         mobil_no =''
    
    # else:
    #     name = 'unknown'
    #     email = 'essalj@gmail.com'
    #     mobil_no= 4527285100
    return name, email, mobil_no

In [27]:
#Task send email and sms
import smtplib, ssl
def serverSendEmail(to, content):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login('essalj@gmail.com', 'Primus9558g')
    server.sendmail('essalj@gmail.com', to, content)
    server.close()
    #sendEmail('essalj@gmail.com','Hejsa. Sikke en test.')


def sendMessage(query, media):
    log = ''
    log +=  media + ' ' + query
    
    name, email, mobil_no = get_receiver(query)
    
    if name=='':
        name = 'unknown'
        email = 'essalj@gmail.com'
        mobil_no = 4527285100
    log += name + ' ' + str(mobil_no) + ' ' + email
    
    content = 'This is a test message'
    #content = take_dictate()
    log += ' ' + content
    
    if media=='email':
        log += media
        try:
            serverSendEmail(email, content)
            speak("Email has been sent to " + name)
        except Exception as e:
            print(e)
            speak("Sorry. I am not able to send this email")

    if media=='sms':
        log += media
        try:
            send_sms.send_sms(mobil_no, 'ROCKY', content)
            speak("Text has been sent to " + name)
        except Exception as e:
            print(e)
            speak("Sorry. I am not able to send the text")

    print(log)
  
# test
# query = 'send text to self'
# media = 'sms'
# sendMessage(query, media)

In [28]:
#Task ask Wolfram
def askWolfram(query):
    #speak('What do you want to know?')
    #question=takeCommand()

    #question = "what is the population of sweden?"
    app_id="Stones"
    client = wolframalpha.Client('4UE782-RAV5EAWPT9')
    res = client.query(question)
    answer = next(res.results).text
    speak("According to wolfram...")
    speak(answer)
    print(answer)
    #askWolfram()

In [29]:
def greetMe():
    hour = int(datetime.datetime.now().hour)
    minute = int(datetime.datetime.now().minute)
    if hour>=0 and hour<12:
        speak("Good Morning!")
    elif hour>=12 and hour<18:
        speak("Good Afternoon!")   
    else:
        speak("Good Evening!") 
    speak("the time is " + str(hour) + ' ' + str(minute))
    
# greetMe()

In [32]:
def what_can_i_say():
    show_commands()


In [34]:
#text stats
def phrase_stats(phrase):
    words = len(phrase.split())
    letters = len(phrase.replace(" ",""))
    return words, letters
# query="search google for blue dogs"
# phrase_stats(query)

# wake up call

In [46]:
def listen(query):

    print("listen(query): ", query)
    #t0 = time.time()
    #speak("How can I help you?")
    #query = takeCommand()
    action_performed = 0

    #0    
    if 'note to self' in query or 'remember this'in query and action_performed==0:
        action_performed = 1
        note_txt = take_dictate()
        note_txt = query
        date = datetime.datetime.now()
        file_name = str(date).replace(':','-') + '-note.txt'
        with open(file_name, "w") as f:
            f.write(note_txt)
        subprocess.Popen('notepad.exe',file_name)
        speak("note created")
        #note_audio = audio
#        print(note)
        time.sleep(3)

    #1
    if 'wikipedia' in query and action_performed==0:  #if wikipedia found in the query
        action_performed = 1
        speak('Searching Wikipedia...')
        query = query.replace("wikipedia", "")
        results = wikipedia.summary(query, sentences=2) 
        speak("According to Wikipedia")
        print(results)
        speak(results)
        time.sleep(3)


    if 'wolfram' in query and action_performed==0:  #if wolfram found in the query
        action_performed = 1
        speak('Searching wolfram...')
        query = query.replace("wolfram", "")
        wolfram(query)
        print(results)
        speak(results)
        time.sleep(3)
    
    
    #8
    if 'ask' in query and action_performed==0:
        action_performed = 1
        askWolfram(query)


#2
    if 'open youtube' in query and action_performed==0:
        action_performed = 1
        webbrowser.get('chrome').open("youtube.com")
        speak("Youtube is ready for you now")
        time.sleep(3)


    #3
    if 'search google' in query and action_performed==0:
        action_performed = 1
        speak("what do I look for?")
        phrase = query.replace('search google',"")
        words, letters = phrase_stats(phrase)
        speak("searching google for " + phrase)
        time.sleep(3)

        #"takeCommand()
        url = "https://www.google.com/search?query=" + phrase
        webbrowser.get('chrome').open(url)
        speak("Google search results are ready for you now")
        time.sleep(3)


    #4
    if 'the time' in query or 'what time' in query and action_performed==0:
        action_performed = 1
        strTime = datetime.datetime.now().strftime("%H:%M:%S")    
        speak(f"The time is {strTime}")
        time.sleep(3)

    #5
    lif 'send email' in query  and action_performed==0':
        action_performed = 1
        sendMessage(query, 'email')
        time.sleep(3)

    
    if 'send text' in query and action_performed==0:
        action_performed = 1
        sendMessage(query,'sms')
        time.sleep(3)

    #6
    if 'the news' in query and action_performed==0:
        action_performed = 1
        url_news = "https://www.dr.dk/drtv/se/tv-avisen-morgen_229629" #dr tv-avisen play direct
        webbrowser.get('chrome').open(url_news)
        speak("The news is playing now")


    if "screenshot" in query and action_performed==0:
        action_performed = 1
        file_name = 'screen_shot_assistant'
        screen_shot("test_fil")

    if ('can i say' in query or 'help me' in query) and action_performed==0:
        action_performed = 1
        what_can_i_say()

    if 'you listening' in query and action_performed==0:
        action_performed = 1
        speak("Discretely..... but yes")

    return action_performed
        
    

SyntaxError: invalid syntax (<ipython-input-46-589eb6901c97>, line 84)

In [43]:
cmd_wake_up_call = 'hello rocky'


t_init = time.time()
action_performed = 0

print("")
while True:
    query = takeCommand()
    t_last_query = time.time()
    
    if query.count(cmd_wake_up_call)>0:
        # speak("Rolling and ready.")
        #time.sleep(1)
        pass_on_query = query.replace(cmd_wake_up_call,' ')
        print("pass on query: ", pass_on_query)
        action_performed = listen(pass_on_query)

        if action_performed == 0:
            greetMe()
            speak("how can I help you?")
            query = takeCommand()
            listen(query)

    elif 'rocky' in query and 'listening' in query:
        speak("Nope.....or maybe just a little")
        time.sleep(3)

    elif 'goodbye rocky' in query:
        speak("goodbye")
        time.sleep(3)
        break           
    



Listening...
Recognizing...
User said: hello Rocky are you listening

pass on query:    are you listening


NameError: name 'listen' is not defined

In [44]:
query

'hello rocky are you listening'

In [45]:
listen()

NameError: name 'listen' is not defined